In [20]:
# ==== Import libraries ====
import math
import random
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)


In [21]:
import csv

input_filename = "GenomeCRISPR_full05112017_brackets.csv"
output_filename = 'GenomeCRISPR_+_strands.csv'
strand_column_index = 3
sequence_column_index = 7

compliments = {"A":"T","T":"A","C":"G","G":"C","N":"N"}

def revcomp(s: str) -> str:
    return "".join(compliments[b] for b in reversed(s))

with open(input_filename, 'r', newline='') as infile, \
     open(output_filename, 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    header = next(reader)
    writer.writerow(header)

    for row in reader:
        if row[strand_column_index] == "-":
            sequence = row[sequence_column_index]
            row[sequence_column_index] = revcomp(sequence)
            row[strand_column_index] = "+"
        writer.writerow(row)

In [17]:
DATA_CSV = "GenomeCRISPR_+_strands.csv"
SEQ_LEN  = 23
VAL_FRAC = 0.10
TEST_FRAC= 0.10


seq_col   = "sequence"
cell_col  = "cellline"
phen_col  = "condition"   # phenotype
chr_col   = "chr"
target_col= "log2fc"      # target



df = pd.read_csv(DATA_CSV)
missing = [c for c in [seq_col, cell_col, phen_col, chr_col, target_col] if c not in df.columns]
if missing:
    raise KeyError(f"Missing expected columns: {missing}. Got: {list(df.columns)}")

df = df[[seq_col, cell_col, phen_col, chr_col, target_col]].copy()
df = df.dropna(subset=[seq_col, cell_col, phen_col, chr_col, target_col])

df[seq_col] = df[seq_col].astype(str).str.upper().str.strip()
df = df[df[seq_col].str.len() == SEQ_LEN]
df = df[df[seq_col].str.match(r"^[ACGT]+$")]

cell_codes, cell_uniques = pd.factorize(df[cell_col].astype(str).str.strip(), sort=True)
phen_codes, phen_uniques = pd.factorize(df[phen_col].astype(str).str.strip(), sort=True)
chr_codes,  chr_uniques  = pd.factorize(df[chr_col].astype(str).str.strip(),  sort=True)

n_cell, n_ph, n_chr = len(cell_uniques), len(phen_uniques), len(chr_uniques)

# One-hot the 23-mer sequences
BASE2IDX = {"A":0, "C":1, "G":2, "T":3}
def onehot_batch(seqs, L=SEQ_LEN):
    N = len(seqs)
    X = np.zeros((N, 4, L), dtype=np.float32)
    for i, s in enumerate(seqs):
        for j, ch in enumerate(s):
            X[i, BASE2IDX[ch], j] = 1.0
    return X

X_seq = onehot_batch(df[seq_col].tolist())
X_cell = cell_codes.astype(np.int64)
X_ph   = phen_codes.astype(np.int64)
X_chr  = chr_codes.astype(np.int64)
y      = df[target_col].astype(np.float32).to_numpy()

# Simple random split
idx_all = np.arange(len(df))
idx_train, idx_test = train_test_split(idx_all, test_size=TEST_FRAC, random_state=42)
idx_train, idx_val  = train_test_split(idx_train, test_size=VAL_FRAC/(1-TEST_FRAC), random_state=42)

def take(a, idx): return a[idx]
Xtr_seq, Xva_seq, Xte_seq = take(X_seq, idx_train), take(X_seq, idx_val), take(X_seq, idx_test)
Xtr_cel, Xva_cel, Xte_cel = take(X_cell, idx_train), take(X_cell, idx_val), take(X_cell, idx_test)
Xtr_ph,  Xva_ph,  Xte_ph  = take(X_ph,  idx_train), take(X_ph,  idx_val), take(X_ph,  idx_test)
Xtr_chr, Xva_chr, Xte_chr = take(X_chr, idx_train), take(X_chr, idx_val), take(X_chr, idx_test)
y_tr,    y_va,    y_te    = take(y,     idx_train), take(y,     idx_val), take(y,     idx_test)

print(f"train={len(idx_train)}  val={len(idx_val)}  test={len(idx_test)}")
print(f"cells={n_cell}  phenotypes={n_ph}  chrs={n_chr}")


NameError: name 'os' is not defined

In [14]:
class MetaDataCNN(nn.Module):
    def __init__(
        self,
        n_cell: int,
        n_ph: int,
        n_chr: int,
        out_channels: int = 16,
        kernel_size: int = 5,
        cell_emb_dim: int = 32,
        ph_emb_dim: int = 8,
        chr_emb_dim: int = 16,
    ):
        super().__init__()
        self.conv = nn.Conv1d(4, out_channels, kernel_size=kernel_size, padding=0)
        self.cell_emb = nn.Embedding(n_cell, cell_emb_dim)
        self.ph_emb   = nn.Embedding(n_ph, ph_emb_dim)
        self.chr_emb  = nn.Embedding(n_chr, chr_emb_dim)
        feat_dim = out_channels + cell_emb_dim + ph_emb_dim + chr_emb_dim
        self.head = nn.Linear(feat_dim, 1)
        

    def forward(self, seq4x23, cell_id, ph_id, chr_id):
        x = self.conv(seq4x23)
        x = F.relu(x)
        x = F.adaptive_max_pool1d(x, 1).squeeze(-1)
        e_cell = self.cell_emb(cell_id)
        e_ph = self.ph_emb(ph_id)
        e_chr  = self.chr_emb(chr_id)
        feats = torch.cat([x, e_cell, e_ph, e_chr], dim=1)  # (B, feat_dim)

        return self.head(feats).squeeze(-1)
        

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = MetaDataCNN(
    n_cell=n_cell,
    n_ph=n_ph,
    n_chr=n_chr,
    out_channels=16,
    kernel_size=5,
    cell_emb_dim=32,
    ph_emb_dim=8,
    chr_emb_dim=16,
).to(device)
print("params:", sum(p.numel() for p in model.parameters()))

cuda
params: 18937


In [15]:
criterion = nn.MSELoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 3
BATCH  = 256

def iter_minibatches(indexes, batch_size=256, shuffle=True):
    idx = np.asarray(indexes)
    if shuffle:
        rng = np.random.default_rng(42)
        rng.shuffle(idx)
    for start in range(0, len(idx), batch_size):
        mb = idx[start:start+batch_size]
        yield (
            torch.from_numpy(X_seq[mb]).to(device),           # (B, 4, 23) float32
            torch.from_numpy(X_cell[mb]).long().to(device),   # (B,) int64
            torch.from_numpy(X_ph[mb]).long().to(device),     # (B,) int64
            torch.from_numpy(X_chr[mb]).long().to(device),    # (B,) int64
            torch.from_numpy(y[mb]).to(device),               # (B,) float32
        )

for epoch in range(1, EPOCHS + 1):
    # ---- train ----
    model.train()
    train_sum, n_train = 0.0, 0
    for seq, cl, ph, ch, tgt in iter_minibatches(idx_train, batch_size=BATCH, shuffle=True):
        pred = model(seq, cl, ph, ch)
        loss = criterion(pred, tgt)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_sum += loss.item() * tgt.size(0)
        n_train   += tgt.size(0)

    # ---- val ----
    model.eval()
    val_sum, n_val = 0.0, 0
    with torch.no_grad():
        for seq, cl, ph, ch, tgt in iter_minibatches(idx_val, batch_size=BATCH, shuffle=False):
            pred = model(seq, cl, ph, ch)
            loss = criterion(pred, tgt)
            val_sum += loss.item() * tgt.size(0)
            n_val   += tgt.size(0)

    print(f"Epoch {epoch:02d} | train MSE: {train_sum/n_train:.4f} | val MSE: {val_sum/n_val:.4f}")

Epoch 01 | train MSE: 0.6411 | val MSE: 0.6386
Epoch 02 | train MSE: 0.6393 | val MSE: 0.6368
Epoch 03 | train MSE: 0.6389 | val MSE: 0.6369


In [16]:
def mse_doc(yhat, y):
    # (1/n) * sum (yi - yhat_i)^2
    yhat = np.asarray(yhat, dtype=np.float64)
    y    = np.asarray(y,    dtype=np.float64)
    n = y.size
    return float(np.sum((y - yhat)**2) / n)

def pearson_doc(x, y):
    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    x = x - x.mean()
    y = y - y.mean()
    denom = np.sqrt(np.sum(x*x) * np.sum(y*y))
    return float(np.sum(x*y) / denom) if denom != 0 else 0.0

def _ranks_avg(a):
    # average ranks (1..n), stable; supports ties
    a = np.asarray(a, dtype=np.float64)
    order = np.argsort(a, kind="mergesort")
    ranks = np.empty_like(order, dtype=np.float64)
    sa = a[order]
    diff = np.concatenate(([True], sa[1:] != sa[:-1], [True]))
    idx = np.flatnonzero(diff)
    for s, e in zip(idx[:-1], idx[1:]):
        ranks[order[s:e]] = 0.5 * (s + e - 1) + 1.0
    return ranks

def spearman_doc(x, y):
    # ρ_s = 1 − (6 Σ d_i^2)/(n(n^2 − 1)), with d_i = rank(x_i) − rank(y_i)
    rx = _ranks_avg(x)
    ry = _ranks_avg(y)
    d  = rx - ry
    n  = rx.size
    denom = n * (n * n - 1.0)
    return float(1.0 - (6.0 * np.sum(d * d)) / denom) if denom != 0.0 else 0.0

# -- accuracy as a decimal --
def accuracy_direction(yhat, y):
    # fraction where predicted and true share the same sign (no rounding/percent)
    yhat = np.asarray(yhat, dtype=np.float64)
    y    = np.asarray(y,    dtype=np.float64)
    return float(np.mean((yhat >= 0) == (y >= 0)))

@torch.no_grad()
def preds_and_trues(indexes, batch_size=256):
    model.eval()
    ps, ys = [], []
    for seq, cl, ph, ch, tgt in iter_minibatches(indexes, batch_size=batch_size, shuffle=False):
        out = model(seq, cl, ph, ch)
        ps.append(out.detach().cpu().numpy())
        ys.append(tgt.detach().cpu().numpy())
    return np.concatenate(ps), np.concatenate(ys)

def eval_split(indexes):
    yhat, y = preds_and_trues(indexes, batch_size=256)
    return {
        "MSE": mse_doc(yhat, y),
        "Pearson": pearson_doc(yhat, y),
        "Spearman": spearman_doc(yhat, y),
        "Accuracy": accuracy_direction(yhat, y),
    }

# --- print results ---
print("Validation:", eval_split(idx_val))
print("Test:",       eval_split(idx_test))

Validation: {'MSE': 0.6368973684840408, 'Pearson': 0.1960569319486082, 'Spearman': 0.15783775394039234, 'Accuracy': 0.5328129566673294}
Test: {'MSE': 0.6395359338009984, 'Pearson': 0.19454467709202422, 'Spearman': 0.15717420077950828, 'Accuracy': 0.5327635211556827}
